In [123]:
#this line tells to plot the matplotlib graphs inline!

%matplotlib inline

import requests
import bs4
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [12]:
req = requests.get("https://www.fbi.gov/about-us/cjis/ucr/crime-in-the-u.s/2013/crime-in-the-u.s.-2013/tables/1tabledatadecoverviewpdf/table_1_crime_in_the_united_states_by_volume_and_rate_per_100000_inhabitants_1994-2013.xls").text

In [13]:
soup = bs4.BeautifulSoup(req, "html.parser")

In [143]:
table = soup.find("table", "data")
tr = table.find("thead").find("tr")
#extract the column names from the table 
columns = [attr.text for attr in tr.find_all("th")]
#remove the /n from the column names
cleaned_columns = [re.sub('\n', "", col) for col in columns]
cleaned_columns = [re.sub('[1-9]', "", col) for col in cleaned_columns]
cleaned_columns

['Year',
 'Population',
 'Violentcrime',
 'Violent crime rate ',
 'Murder andnonnegligent manslaughter',
 'Murder and nonnegligent manslaughter rate ',
 'Rape(legacydefinition)',
 'Rape(legacydefinition)rate',
 'Robbery',
 'Robbery rate ',
 'Aggravated assault',
 'Aggravated assault rate ',
 'Property crime',
 'Property crime rate ',
 'Burglary',
 'Burglary rate ',
 'Larceny-theft',
 'Larceny-theft rate ',
 'Motor vehicle theft',
 'Motor vehicle theft rate ']

In [144]:
#get all the data rows
data_rows = table.find("tbody").find_all("tr")
data_rows = data_rows[:19]
list_data = []
years = []
for row in data_rows:
    data_in_row = row.find_all("td")
    years.append((row.find("th")).text)
    act_data = [data.text for data in data_in_row]
    list_data.append(act_data)
#coolest list comprehention written!!, cleaned the data
cleaned_data = [[re.sub('\n', "", th) for th in rows] for rows in list_data] 
#also remove all the commas
cleaned_data = [[re.sub(',', "", th) for th in rows] for rows in cleaned_data]    
#the only task left is to append year in the starting of list in every data
cleaned_data[:1]

[['260327021',
  '1857670',
  '713.6',
  '23326',
  '9.0',
  '102216',
  '39.3',
  '618949',
  '237.8',
  '1113179',
  '427.6',
  '12131873',
  '4660.2',
  '2712774',
  '1042.1',
  '7879812',
  '3026.9',
  '1539287',
  '591.3']]

In [145]:
#But first we have to clean the data and remove idiot datapoints 
#remove /n
years = [re.sub('\n', "", y) for y in years]
years = [y[:4] for y in years]
#cleaned_data[0].insert(0, years[0])
for i in range(19):
    cleaned_data[i].insert(0, years[i])
#this is the final data    
cleaned_data[:1]

[['1994',
  '260327021',
  '1857670',
  '713.6',
  '23326',
  '9.0',
  '102216',
  '39.3',
  '618949',
  '237.8',
  '1113179',
  '427.6',
  '12131873',
  '4660.2',
  '2712774',
  '1042.1',
  '7879812',
  '3026.9',
  '1539287',
  '591.3']]

In [146]:
cleaned_data = np.asarray(cleaned_data)
data_org = pd.DataFrame(cleaned_data, columns = cleaned_columns)

## HELL YEAH!!

In [147]:
data_org.head()

,Year,Population,Violentcrime,Violent crime rate,Murder andnonnegligent manslaughter,Murder and nonnegligent manslaughter rate,Rape(legacydefinition),Rape(legacydefinition)rate,Robbery,Robbery rate,Aggravated assault,Aggravated assault rate,Property crime,Property crime rate,Burglary,Burglary rate,Larceny-theft,Larceny-theft rate,Motor vehicle theft,Motor vehicle theft rate
0,1994,260327021,1857670,713.6,23326,9.0,102216,39.3,618949,237.8,1113179,427.6,12131873,4660.2,2712774,1042.1,7879812,3026.9,1539287,591.3
1,1995,262803276,1798792,684.5,21606,8.2,97470,37.1,580509,220.9,1099207,418.3,12063935,4590.5,2593784,987.0,7997710,3043.2,1472441,560.3
2,1996,265228572,1688540,636.6,19645,7.4,96252,36.3,535594,201.9,1037049,391.0,11805323,4451.0,2506400,945.0,7904685,2980.3,1394238,525.7
3,1997,267783607,1636096,611.0,18208,6.8,96153,35.9,498534,186.2,1023201,382.1,11558475,4316.3,2460526,918.8,7743760,2891.8,1354189,505.7
4,1998,270248003,1533887,567.6,16974,6.3,93144,34.5,447186,165.5,976583,361.4,10951827,4052.5,2332735,863.2,7376311,2729.5,1242781,459.9


In [148]:
data_org.dtypes

Year                                          object
Population                                    object
Violentcrime                                  object
Violent crime rate                            object
Murder andnonnegligent manslaughter           object
Murder and nonnegligent manslaughter rate     object
Rape(legacydefinition)                        object
Rape(legacydefinition)rate                    object
Robbery                                       object
Robbery rate                                  object
Aggravated assault                            object
Aggravated assault rate                       object
Property crime                                object
Property crime rate                           object
Burglary                                      object
Burglary rate                                 object
Larceny-theft                                 object
Larceny-theft rate                            object
Motor vehicle theft                           

In [151]:
#convert the data types of the data accordingly 
#if the data is rate, convert it to float
#else convert it to integer
for col in cleaned_columns:
    if 'rate' in col:
        data_org[col] = data_org[col].astype(float)
    else:    
        data_org[col] = data_org[col].astype(int)
data_org.dtypes

Year                                            int64
Population                                      int64
Violentcrime                                    int64
Violent crime rate                            float64
Murder andnonnegligent manslaughter             int64
Murder and nonnegligent manslaughter rate     float64
Rape(legacydefinition)                          int64
Rape(legacydefinition)rate                    float64
Robbery                                         int64
Robbery rate                                  float64
Aggravated assault                              int64
Aggravated assault rate                       float64
Property crime                                  int64
Property crime rate                           float64
Burglary                                        int64
Burglary rate                                 float64
Larceny-theft                                   int64
Larceny-theft rate                            float64
Motor vehicle theft         

In [152]:
data_org.head()

,Year,Population,Violentcrime,Violent crime rate,Murder andnonnegligent manslaughter,Murder and nonnegligent manslaughter rate,Rape(legacydefinition),Rape(legacydefinition)rate,Robbery,Robbery rate,Aggravated assault,Aggravated assault rate,Property crime,Property crime rate,Burglary,Burglary rate,Larceny-theft,Larceny-theft rate,Motor vehicle theft,Motor vehicle theft rate
0,1994,260327021,1857670,713.6,23326,9.0,102216,39.3,618949,237.8,1113179,427.6,12131873,4660.2,2712774,1042.1,7879812,3026.9,1539287,591.3
1,1995,262803276,1798792,684.5,21606,8.2,97470,37.1,580509,220.9,1099207,418.3,12063935,4590.5,2593784,987.0,7997710,3043.2,1472441,560.3
2,1996,265228572,1688540,636.6,19645,7.4,96252,36.3,535594,201.9,1037049,391.0,11805323,4451.0,2506400,945.0,7904685,2980.3,1394238,525.7
3,1997,267783607,1636096,611.0,18208,6.8,96153,35.9,498534,186.2,1023201,382.1,11558475,4316.3,2460526,918.8,7743760,2891.8,1354189,505.7
4,1998,270248003,1533887,567.6,16974,6.3,93144,34.5,447186,165.5,976583,361.4,10951827,4052.5,2332735,863.2,7376311,2729.5,1242781,459.9


### Now the data has been cleaned, 